# Introduction to `pyClarion`



This tutorial introduces basic concepts and workflows for building cognitive models and artificial agents using the `pyClarion` library.

Familiarity with the Clarion cognitive architecture is recommended but not required. The setting is loosely inspired by the *Wisconsin Card Sorting Task* (WCST), which is used in the assessment of executive function. Familiarity with this task is also not required, it is sufficient to know that the task involves matching cards based on various figural attributes. 

In Clarion, WCST cards may be explicitly represented by chunks encoding their figural attributes. Activation of these attributes in the bottom level of the architecture may then drive partial activation of matching chunks. The present model computes the activations of such chunks based on (micro)feature activations in the bottom level. This is a fairly elementary process that nevertheless illustrates many core features of the library.

## Background

In Clarion, the chunk is the basic unit of explict knowledge. Chunks link chunk nodes in the top level of the architecture to collections of (micro)features in the bottom level of the architecture. These (micro)features are, in turn, represented in the form of dimension-value pairs. Links between a chunk node and its (micro)features are defined by a chunk's *top-down weight vector*, which determines how activations propagate from the chunk node to each of its (micro)feature nodes. 

Activations may also flow in the reverse direction, from (micro)feature nodes to chunk nodes. Propagation in this reverse direction is called *bottom-up activation*. The purpose of the present model is to compute bottom-up activations for chunks representing various WCST cards from the activations of (micro)features representing figural attributes.

For a chunk $c$, its bottom-up activation $s_c^{bu}$ is computed as follows, where $w_{ci}$ designates the top-down dimensional weight linking chunk $c$ to features associated with its $i$-th dimension, $x_i$ is the activation of dimension $i$ of chunk $c$ in the bottom level, and $f$ is a superlinear function of some norm of $w$ (in the present implementation $f(w) = 1 + \sum_i |w_i|$). 
$$
s_c^{bu} = \frac{\sum_i w_{ci} x_i}{f(w)}
$$
It is possible for the chunk $c$ to be associated with multiple dimension-value pairs of a shared dimension $i$. In such cases, the dimensional activation $x_i$ is the maximum of all dimension-value pairs of the chunk $c$ that are associated with dimension $i$. Though, such cases do not arise in the present model.

## Model

Broadly speaking, a `pyClarion` model amounts to a *discrete event simulation*. That is to say, `pyClarion` models are defined by collections of processes that generate sequences of discrete events in continuous time. 
A model's state is encoded as collections of associations between symbolic keys and numerical values. A model's *keyspace* defines the symbolic keys available to a simulation. The implementation of a `pyClarion` simulation may therefore be broken down into four main activities: keyspace definition, model construction, knowledge initialization, and event processing. 

### Keyspace Definition

Keyspace definition is the process of specifying the basic data dimensions that define a simulation environment. 

Keyspaces are typically first defined in an abstract manner and then instantiated as part of model construction. Formally, a model's keyspace consists of a three-level hierarchy of symbolic keys. These keys collectively define basic terms for indexing simulation data. Within a model's keyspace, a key may be classified as a *term*, *sort*, or *family* according to its level (from lowest to highest). A term names an individual data dimension (e.g., an individual feature, parameter, etc.), a sort groups terms that are alike in content (e.g., color terms, shape terms, etc.), and a family groups sorts that play like roles within a simulation (e.g., data, parameters, etc.). 

For the domain of the WCST, symbols may be defined to represent various figural properties such as the color, shape, and number of figures displayed on a card. Furthermore, some additional terms, representing different data roles (inputs, outputs, targets, etc.) may also be needed.  

In [1]:
"""Data hierarchy for the WCST domain."""

from pyClarion import Atom # Represents an atomic term
from pyClarion import Atoms # Represents a sort grouping atomic terms
from pyClarion import Family # Represents a family of terms

class Color(Atoms): 
    """A sort for atomic color terms."""
    red: Atom
    grn: Atom
    blu: Atom

class Shape(Atoms): 
    """A sort for atomic shape terms."""
    circ: Atom
    squr: Atom
    tria: Atom

class Number(Atoms):
    """A sort for atomic number terms."""
    one: Atom
    two: Atom
    three: Atom
    four: Atom

class IO(Atoms):
    """A sort for atomic input/output terms."""
    input: Atom
    output: Atom
    target: Atom

class WCSTData(Family):
    """A family for all data sorts."""
    color: Color
    shape: Shape
    number: Number
    io: IO

### Model Construction

Model construction is the process of initializing and assembling a collection of `pyClarion` component processes that implement the functionality of a desired model. 

The `pyClarion` library defines component processes implementing the main elements of the Clarion cognitive architecture, and it allows models to be constructed in a modular and compositional fashion.

For the present purposes, model construction requires the `Agent`, `Input`, and `ChunkStore` processes and the `WCSTData` keyspace.

In [2]:
"""Agent construction for bottom-up activation model."""

from pyClarion import Agent # Initialzes keyspaces specific to an agent
from pyClarion import Input # Receives activations from external sources
from pyClarion import ChunkStore # Maintains a collection of Clarion chunks
from pyClarion import BottomUp

# Initialize basic data symbols for current simulation
# During initialization, WCSTData() will automatically generate symbols for all 
# terms annotated with a sort or term type.
d = WCSTData() 

# Create a new agent and populate its keyspace with data symbols.
# Agent expects Family instances as kwargs and will automatically populate the 
# keyspace with them.
agent = Agent("agent", d=d)

# Any process objects initialized within this block will be automatically added 
# to the agent.
with agent:  

    # Create an input process to pass in feature activations. The tuple `(d, d)`
    # indicates that this process will receive inputs in the form of 
    # dimension-value pairs constructed by pairing symbols from the family 'd'.
    ipt = Input("ipt", (d, d))

    # Create a chunk store using the family 'd' to house chunk symbols (arg 
    # 'c') and represent feature dimensions and values (args 'd' and 'v').
    chunks = ChunkStore("chunks", c=d, d=d, v=d)

    # Create a bottom up activation process dependent on chunk store.
    bu = ipt >> BottomUp.from_store("bu", chunks) 

Note that a model may contain more processes than were explicitly specified. This is because some processes may depend on or extend the functionality of more elementary processes. 

In [3]:
# List all processes associated with the current model
agent.system.procs

[<Agent 'agent' at 0x7fd83ad16f60>,
 <Input 'ipt' at 0x7fd83ad16060>,
 <ChunkStore 'chunks' at 0x7fd83ad17260>,
 <BottomUp 'bu' at 0x7fd83ad176b0>]

### Knowledge Initialization

Knowledge initialization is the process of defining any prior knowledge available to a model. 

Typically, this involves the specification of user-defined explicit knowledge such as chunks and/or rules using `pyClarion`'s knowledge representation tools. It may also, however, include populating the model with other prior knowledge, such as pretrained neural network weights. 

For the present example, knowledge initialization involves defining some chunks for which to calculate bottom-up activations. Three chunks are defined below, representing three distinct card faces.

In [4]:
"""Chunks for bottom-up activation model"""

# Define short handles for data sorts
io = d.io
color = d.color   
shape = d.shape  
number = d.number 

# Define three chunks. Technically, these chunks are represented as terms.
# Note that dimension-value pairs are constructed by pairing together two terms 
# using the '**' operator. Positive sign indicates a top-down weight of +1.0. 
# Other weights may be assigned by left multiplication with floats.
chunk_defs = [
    # The caret operator can be used to name chunks and rules. 
    "one_blue_triangle" ^ 
    + io.input ** color.blu
    + io.input ** shape.tria
    + io.input ** number.one,

    "one_red_triangle" ^
    + io.input ** color.red
    + io.input ** shape.tria
    + io.input ** number.one,

    "two_green_squares" ^
    + io.input ** color.grn
    + io.input ** shape.squr
    + io.input ** number.two,
]

# Schedule an event to populate the model with the new chunks. This event will 
# bind the terms defined above to the model's keyspace and initialize their 
# top-down and bottom-up weights. The knowledge defined in `chunk_defs` will not 
# be available to the model until this event is executed (during event 
# processing).
agent.system.schedule(chunks.encode(*chunk_defs))

### Event Processing

Event processing is the process of getting a model to generate and process simulation events. 

Events have four primary effects on the state of a simulation: They may (i) update numerical data in process sites, (ii) update the simulation's keyspace, (iii) cause processes to schedule further events, or (iv) advance the simulation clock. Events do not actively perform computations, thus updates may only depend on the state of a model at the time that an event is scheduled.

Scheduled events are maintained in an event queue, which ensures that they are processed in due time. The preceding step has already generated one event which, when processed, will have all of the effects listed above.  


In [5]:
# List all currently scheduled events. Technically, the event queue is 
# implemented as a heap, so events are not guaranteed to be listed in order.
agent.system.queue

[<Event source=chunks.encode time=datetime.timedelta(0) at 0x7fd83ad3a750>]

To facilitate tracking events and other relevant data during the course of a simulation, event logs may be generated using Python's `logging` module.

In [6]:
"""Initialize event logging for simulation"""

import logging
import sys

logger = logging.getLogger("pyClarion.system")
logger.setLevel(logging.DEBUG)
logger.handlers.clear()
logger.addHandler(logging.StreamHandler(sys.stdout))

The present simulation may be completed simply by sending some inputs into the bottom level of the model.

In [7]:
"""Run simulation"""

# Schedule an event to update the input to the model with the following data.
# Positives get +1.0 activation, negatives get -1.0 activation. 
agent.system.schedule(
    ipt.send(    
        + io.input ** color.blu
        + io.input ** shape.tria
        + io.input ** number.one
        - io.input ** number.two
    ) 
)

# The following lines process all events in the current queue.
while agent.system.queue:
    event = agent.system.advance() # Process and return the next event.
    # Can respond to event here (e.g., record data, stop simulation, 
    # communicate with external resources etc.)

event 0x0000 00:00:00.00 096 0 chunks.encode
    Added the following new chunk(s)
    chunk d:chunks:one_blue_triangle
        + io.input ** color.blu
        + io.input ** shape.tria
        + io.input ** number.one
    chunk d:chunks:one_red_triangle
        + io.input ** color.red
        + io.input ** shape.tria
        + io.input ** number.one
    chunk d:chunks:two_green_squares
        + io.input ** color.grn
        + io.input ** shape.squr
        + io.input ** number.two
[(Key('(d,d,d):(chunks,color,color):(nil,red,red)'), [0.0, 1.0]), (Key('(d,d,d):(chunks,color,color):(one_blue_triangle,red,red)'), [0.0, 4.0]), (Key('(d,d,d):(chunks,color,color):(one_red_triangle,red,red)'), [0.0, 4.0]), (Key('(d,d,d):(chunks,color,color):(two_green_squares,red,red)'), [0.0, 4.0]), (Key('(d,d,d):(chunks,color,color):(nil,red,grn)'), [0.0, 1.0]), (Key('(d,d,d):(chunks,color,color):(one_blue_triangle,red,grn)'), [0.0, 4.0]), (Key('(d,d,d):(chunks,color,color):(one_red_triangle,red,grn)'), [0.

## Results

The result of computing bottom-up activations from the given input may be obtained by accessing the main output site of the bottom-up activation process `chunks.bu`. This presents an occasion for a glimpse into the internal representations of a `pyClarion` model.

In [8]:
# The bottom-up output is stored in the data site called bu.main. The 
# current data at a site is accessed via subscripting at index 0. Some sites 
# retain lagged data, which may be accessed by subscripting with the 
# corresponding lag value.
data = bu.main[0]

# Print the outcome of the bottom-up activation process.
# The first line of the result contains some general information, each 
# subsequent line lists key-value associations which, in this case, represent 
# bottom-up activations for each chunk. Consult the event logs for information 
# on the chunk identifiers.
print(data)

NumDict 'd:chunks:?' c=0.0
    d:chunks:one_blue_triangle 0.75
    d:chunks:one_red_triangle  0.5


## Conclusion

The `pyClarion` library presents an integrated suite of tools for implementing cognitive models within the Clarion cognitive architecture (and even beyond). It spans a wide range of computational domains including discrete event simulation, knowledge representation, and numerical computation. 

This tutorial introduces some of the main features and workflows of these tools. Specifically, it illustrates the four main activities involved in model implementation: keyspace definition, model construction, knowledge initialization, and event processing. As such, the tutorial provides preparation for more advanced topics including detailed treatments of knowledge representation, neural networks, and process customization. 

Finally, for readers new to cognitive modeling, it is worth highlighting that there is more to model development than the implementation steps outlined here. Typically, model development requires intensive theoretical and conceptual work that is distinct from, and often more critical than, the work required for implementation. 